In [ ]:
import plotly.graph_objects as go
 
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
import shap
import os
import warnings
warnings.filterwarnings('ignore')


try:
    from google.colab import drive
    colab = True
    drive.mount('/content/drive')
    data_path = '/content/drive/Shared drives/MSD 23 Drive/Datasets'
    output_path = '/content/drive/Shared drives/MSD 23 Drive/Figures'
except:
    colab = False
    data_path = './MSD Datathon Files/Datasets/'
    output_path = './MSD Datathon Files/Datasets/'

print('Data path: ', data_path)
print('Output path: ', output_path)

In [ ]:
#teds_state_year.csv and facility_state_year.csv
name = 'teds_state_year.csv'
file = os.path.join(data_path,  name)
df1 = pd.read_csv(file)

name = 'facility_state_year.csv'
file = os.path.join(data_path,  name)
df2 = pd.read_csv(file)

df = pd.merge(df1, df2, on=['STATE', 'year'])

#teds_state_year_rehab.csv and facility_state_year.csv
name = 'teds_state_year_rehab.csv'
file = os.path.join(data_path,  name)
df1 = pd.read_csv(file)

df_rehab = pd.merge(df1, df2, on=['STATE', 'year'])

#teds_state_year_detox.csv and facility_state_year.csv
name = 'teds_state_year_detox.csv'
file = os.path.join(data_path,  name)
df1 = pd.read_csv(file)

df_detox = pd.merge(df1, df2, on=['STATE', 'year'])


In [ ]:
# features to use as predictors
cont_cols = ['ASSESSMENT', 
             'TESTING', 
             'TRANSITION', 
             'ANCILLARY', 
             'OTHER_SRVC']

binar_cols = ['DETOX',  
              'TREATMT',  
              'HOSPITAL',  
              'SRVC75',  
              'SRVC71',  
              'SRVC108',  
              'SRVC85',  
              'SRVC87',  
              'SRVC86',  
              'OTP',  
              'OPIOIDMAINT',
              'OPIOIDWDRAW',  
              'OPIOIDDETOX',  
              'OPIOIDNAL',  
              'DUI_DWI',  
              'ONLYDUI',  
              'SIGNLANG',  
              'SRVC30',  
              'SRVC34',  
              'SRVC33',  
              'SRVC64',  
              'SRVC63',
              'SRVC62',  
              'SRVC113',  
              'SRVC114',  
              'SRVC115',  
              'SRVC61',  
              'SRVC31',  
              'SRVC32',  
              'SRVC116',  
              'CTYPE4',  
              'CTYPE7',  
              'CTYPERC1',  
              'CTYPERC3', 
              'CTYPERC4',  
              'CTYPE1',  
              'CTYPE6',  
              'CTYPEML',  
              'CTYPEOP',  
              'CTYPE2',  
              'CTYPE3', 
              'FEESCALE',  
              'PAYASST',  
              'EARMARK',
              'REVCHK3',  
              'REVCHK1',  
              'REVCHK8',  
              'REVCHK5',  
              'REVCHK10',  
              'REVCHK15',  
              'REVCHK2',  
              'REVCHK17', 
              'LOC5',  
              'LICEN',  
              'ACCRED',  
              'OWNERSHP_Federal Government',  
              'OWNERSHP_Local, county or community government', 
              'OWNERSHP_Private non-profit organization',     
              'OWNERSHP_Private-for-profit organization',  
              'OWNERSHP_State government',  
              'OWNERSHP_Tribal Government']

features = cont_cols + binar_cols + ['facility_density']

In [ ]:
X = [df[features], df_detox[features], df_rehab[features]]
y_targets = [1- df['REASON_Treatment completed'], 1- df_detox['REASON_Treatment completed'], 1- df_rehab['REASON_Treatment completed']] #Failure of treatments. 

In [ ]:
FI_list = []

In [ ]:
for i in range(0, 3):

  X_train, X_test, y_train, y_test = train_test_split(X[i], y_targets[i], test_size = 0.40, random_state=0)
  # Instantiate random forest regressor
  rf = RandomForestRegressor(n_estimators=200,random_state=0)
  # Use 10-fold cross-val to get estimate of training RMSE
  score = cross_val_score(rf,
                          X_train,
                          y_train,
                          cv=10,
                          scoring='neg_mean_squared_error')
  print("Cross-validated negative root mean squared error score: ", np.sqrt(-score.mean()))
  # Fit the actual regressor
  rf.fit(X_train, y_train)
  # predict X_test
  y_pred = rf.predict(X_test)
  # get test set RMSE
  print('Evaluation on held-out test-set: ', np.sqrt(mean_squared_error(y_test, y_pred)))

  sorted_indices = rf.feature_importances_.argsort()    
  #Sergio: storing feature importance.
  FI_list.append([features[index] for index in sorted_indices])
  FI_list.append(rf.feature_importances_[sorted_indices])

  plt.figure(figsize = (3, 10))

  plt.barh([features[index] for index in sorted_indices], rf.feature_importances_[sorted_indices])
  plt.xlabel('Importance')
  plt.xlim([0, 0.2])

  plt.show()


    
  # Get SHAP values from rf model
  explainer = shap.Explainer(rf)
  # Explain model
  shap_values = explainer(X[i])
  # Beeswarm plot - SHAP values
  shap.summary_plot(shap_values, X)

  # Plot y_pred vs y_test
  plt.scatter(y_test, y_pred)
  plt.ylabel('Predictions')
  plt.xlabel('Ground Truth')
  plt.show()


In [ ]:
#Sergio's cell
FI_need_df = pd.DataFrame(FI_list).transpose()
FI_need_df.columns = ['Features REASON_Treatment fail', 'Importance REASON_Treatment fail', 
                      'Features REASON_Treatment fail detox', 'Importance REASON_Treatment fail detox', 
                      'Features REASON_Treatment fail rehab', 'Importance REASON_Treatment fail rehab']

name = 'treatment_failure_feature_importance.csv'
file = os.path.join(data_path, name)

FI_need_df.to_csv(file, index=False)